#### _Matin's Lab Rotation 3 - BFCO_ | [NEXT](03_community_dFBA_growth_objective.ipynb) | [PREVIOUS](01-1_simulation_isobutyric_strain.ipynb)
# 0X. Test acetate exchange

In [1]:
from cameo import load_model
from cameo import fba
from escher import Builder
import pandas as pd
import cobra

/home/matinnu/anaconda3/envs/lab_rotation_3/lib/python3.7/site-packages/sympy/__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.singleton with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.singleton instead.
See https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()


In [2]:
model = load_model('iML1515')
model.solver = 'gurobi'
model

Using license file /home/matinnu/gurobi.lic
Academic license - for non-commercial use only


Name,iML1515
Memory address,0x07fdb30ebd610
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [3]:
mutant1 = load_model('result/iML1515_xylitol_strain.xml')
mutant1.solver = 'gurobi'
mutant1

Name,iML1515
Memory address,0x07fdb2f994990
Number of metabolites,1880
Number of reactions,2715
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [4]:
mutant2 = load_model('result/iML1515_isobutyric_strain.xml')
mutant2.solver = 'gurobi'
mutant2

Name,iML1515
Memory address,0x07fdb2892c190
Number of metabolites,1881
Number of reactions,2716
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [5]:
media_components = [["EX_xyl__D_e", 5.0],
                    ["EX_glc__D_e", 5.0],
                   ]

medium = mutant1.medium
for i in media_components:
    medium[i[0]] = i[1]
    mutant1.medium = medium
#print(model.medium)

mutant2.medium = mutant1.medium
model.medium = mutant2.medium

In [21]:
reaction_1 = mutant1.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M')
reaction_2 = mutant1.reactions.get_by_id('EX_xylt_e')
reaction_3 = mutant1.reactions.get_by_id('EX_ac_e')
mutant1_new_objective = {reaction_1: 0.8, reaction_2:0.18, reaction_3:0.02}
mutant1.objective = mutant1_new_objective
mutant1.reactions.XYLK.knock_out()
mutant1.genes.b0733.knock_out()
#cobra.io.write_sbml_model(mutant1, "result/model_for_consortia/iML1515_mutant1.xml")

In [7]:
#reaction_1 = mutant2.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M')
reaction_4 = mutant2.reactions.get_by_id('EX_isobuta_e')
mutant2_new_objective = {reaction_1: 0.9, reaction_4:0.1}
mutant2.objective = mutant2_new_objective
#cobra.io.write_sbml_model(mutant2, "result/model_for_consortia/iML1515_mutant2.xml")

In [8]:
solution0 = model.optimize()

In [22]:
solution1 = mutant1.optimize()

In [10]:
ac_e = mutant1.reactions.EX_ac_e.summary().to_frame().values[0][0]
ac_e
media_components = [["EX_ac_e", ac_e]]
medium = mutant2.medium
for i in media_components:
    medium[i[0]] = i[1]
    mutant2.medium = medium

In [11]:
solution2 = mutant2.optimize()

In [12]:
builder = Builder(
    map_json='map/iJO1366.Central metabolism_rotation3.json',
    reaction_data=solution0.fluxes,
    #reaction_data=solution.fluxes,
)
builder.save_html('result/flux_map/model.html')
builder

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': -0.020855913639397615, 'SHK3Dr': 0.3…

In [23]:
builder = Builder(
    map_json='map/iJO1366.Central metabolism_rotation3.json',
    reaction_data=solution1.fluxes,
    #reaction_data=solution.fluxes,
)
builder.save_html('result/flux_map/mutant1.html')
builder

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': 0.0, 'SHK3Dr': 0.05434011884212355, …

In [14]:
#print(mutant1.reactions.BIOMASS_Ec_iML1515_core_75p37M.summary())
mutant1.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0007421,0,0.00%
cl_e,EX_cl_e,0.0007421,0,0.00%
cobalt2_e,EX_cobalt2_e,3.565E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001011,0,0.00%
fe2_e,EX_fe2_e,0.00229,0,0.00%
glc__D_e,EX_glc__D_e,5,6,54.55%
k_e,EX_k_e,0.02783,0,0.00%
mg2_e,EX_mg2_e,0.001237,0,0.00%
mn2_e,EX_mn2_e,9.852E-05,0,0.00%
mobd_e,EX_mobd_e,9.981E-07,0,0.00%


In [15]:
builder = Builder(
    map_json='map/iJO1366.Central metabolism_rotation3.json',
    reaction_data=solution2.fluxes,
    #reaction_data=solution.fluxes,
)
builder.save_html('result/flux_map/mutant2.html')
builder

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': 0.0, 'SHK3Dr': 0.07328249908683064, …

In [16]:
mutant2.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,7.847,2,34.34%
ca2_e,EX_ca2_e,0.001001,0,0.00%
cl_e,EX_cl_e,0.001001,0,0.00%
cobalt2_e,EX_cobalt2_e,4.807E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001363,0,0.00%
fe2_e,EX_fe2_e,0.003088,0,0.00%
glc__D_e,EX_glc__D_e,5,6,65.66%
h_e,EX_h_e,0.3611,0,0.00%
k_e,EX_k_e,0.03753,0,0.00%
mg2_e,EX_mg2_e,0.001668,0,0.00%


In [20]:
data0 =[solution0.fluxes.BIOMASS_Ec_iML1515_core_75p37M,
        solution0.fluxes.EX_co2_e, 
        solution0.fluxes.EX_ac_e,
        #solution1.fluxes.EX_xylt_e
       ]
print(data0)

data1 =[solution1.fluxes.BIOMASS_Ec_iML1515_core_75p37M,
        solution1.fluxes.EX_co2_e, 
        solution1.fluxes.EX_ac_e,
        solution1.fluxes.EX_xylt_e
       ]
print(data1)

data2 =[solution2.fluxes.BIOMASS_Ec_iML1515_core_75p37M,      
        solution2.fluxes.EX_co2_e,
        solution2.fluxes.EX_ac_e,
        solution2.fluxes.EX_isobuta_e,
]
print(data2)

[0.7970615928838041, 22.28428331307737, 0.0]
[0.14258045083588997, 8.454358571313, 7.846684468232387, 5.0]
[0.19228246056174222, 14.925806342309938, -7.846684468232434, 5.718812733429791]
